In [ ]:
! pip install deap

In [ ]:
import random
import time
import array
import copy

from deap import base
from deap import creator
from deap import tools
from deap import algorithms
from deap.benchmarks import griewank

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib import cm
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import axes3d
from scipy.stats import norm, multivariate_normal
from IPython import display
import math

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
def plot_individual(individual, ax=None):
    'Plots an ES indiviual as center and 3*sigma ellipsis.'
    cov = np.eye(len(individual)) * individual.strategy
    plot_cov_ellipse(individual, cov, volume=0.99, alpha=0.56, ax=ax)
    if ax:
        ax.scatter(individual[0], individual[1], 
                    marker='+', color='k', zorder=100)
    else:
        plt.scatter(individual[0], individual[1], 
                    marker='+', color='k', zorder=100)

    
def plot_population(pop, gen=None, max_gen=None, ax=None):
    if gen:
        plt.subplot(max_gen, 1, gen)
        
    for ind in pop:
        plot_individual(ind, ax)

def plot_problem_controur(problem, bounds, optimum=None,
                          resolution=100., cmap=cm.viridis_r, 
                          rstride=1, cstride=10, linewidth=0.15,
                          alpha=0.65, ax=None):
    'Plots a given deap benchmark problem as a countour plot'
    (minx,miny),(maxx,maxy) = bounds
    x_range = np.arange(minx, maxx, (maxx-minx)/resolution)
    y_range = np.arange(miny, maxy, (maxy-miny)/resolution)
    
    X, Y = np.meshgrid(x_range, y_range)
    Z = np.zeros((len(x_range), len(y_range)))
    
    for i in range(len(x_range)):
        for j in range(len(y_range)):
            Z[i,j] = problem((x_range[i], y_range[j]))[0]
    
    if not ax:
        fig = plt.figure(figsize=(6,6))
        ax = fig.gca()
        ax.set_aspect('equal')
        ax.autoscale(tight=True)
    
    cset = ax.contourf(X, Y, Z, cmap=cmap, alpha=alpha)
    
    if optimum:
        ax.plot(optimum[0], optimum[1], 'bx', linewidth=4, markersize=15)

def plot_cov_ellipse(pos, cov, volume=.99, ax=None, fc='lightblue', ec='darkblue', alpha=1, lw=1):
    ''' Plots an ellipse that corresponds to a bivariate normal distribution.
    Adapted from http://www.nhsilbert.net/source/2014/06/bivariate-normal-ellipse-plotting-in-python/'''
    from scipy.stats import chi2
    from matplotlib.patches import Ellipse

    cov = [[cov[0][0], cov[0][-1]],[cov[-1][0], cov[-1][-1]]]

    def eigsorted(cov):
        vals, vecs = np.linalg.eigh(cov)
        order = vals.argsort()[::-1]
        return vals[order], vecs[:,order]

    if ax is None:
        ax = plt.gca()

    vals, vecs = eigsorted(cov)
    theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))

    kwrg = {'facecolor':fc, 'edgecolor':ec, 'alpha':alpha, 'linewidth':lw}

    # Width and height are "full" widths, not radius
    width, height = 2 * np.sqrt(chi2.ppf(volume,2)) * np.sqrt(vals)
    ellip = Ellipse(xy=pos, width=width, height=height, angle=theta, **kwrg)
    ax.add_artist(ellip)

X = []
Y = []
Z = []

def l_show(x, y):
  return griewank([x,y])[0]

def startPlot():
  global X, Y, Z
  rastrigin_vectorized = np.vectorize(l_show)

  x = np.linspace(-10, 10, 60)
  y = np.linspace(-10, 10, 60)

  X, Y = np.meshgrid(x, y)
  Z = rastrigin_vectorized(X, Y)
  plt.figure()

def updatePlot(population):
  global X, Y, Z
  plt.clf()
  x1 = [ind[random.randint(0, len(ind)-1)] for ind in population]
  x1 = [a if a > -10 and a < 10 else max(-10, min(a, 10)) for a in x1]
  x2 = [ind[random.randint(0, len(ind)-1)] for ind in population]
  x2 = [a if a > -10 and a < 10 else max(-10, min(a, 10)) for a in x2]
  plt.contour(X, Y, Z)
  plt.scatter(x1, x2, color='red')
  display.display(plt.gcf())
  display.clear_output(wait=True)

In [ ]:
# funcao de avaliação com a função griewank
def evalProblem(individual):
  return griewank(individual)

search_space_dims = 10 # we want to plot the individuals so this must be 2

MIN_VALUE, MAX_VALUE = -20., 20.
MIN_STRAT, MAX_STRAT = 0.0000001, 1. 

# Cria no DEAP o tipo de função de fitness (Minizar, Maximizar ou Multiobjetivo)
#A função create () recebe pelo menos dois argumentos, um nome para a classe recém-criada e uma classe base.
#O terceiro argumento indica se o problema é de minimizar (negativo), maximizar (positivo), para múltiplos objetivos esse argumento deve ser uma tupla.
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

# Estratégias evolutivas precisam de uma localização (média)
#O primeiro argumento indica que estamos criando o formato indivíduo, o segundo é a estrutura dos indivíduo, o argumento typecode 
#está relacionado com o formato de array e indica que serão armazenados somente doubles, o próximo argumento é a vinculação com o objeto do fitness
#Por fim, ao trabalhar com ES é necessário que o campo "strategy" seja adicionado ao indivíduo 
creator.create("Individual", array.array, typecode='d', 
               fitness=creator.FitnessMin, strategy=None)
# ...e um valor do parâmetro estratégia.
creator.create("Strategy", array.array, typecode="d")

def init_univariate_es_ind(individual_class, strategy_class,
                           size, min_value, max_value, 
                           min_strat, max_strat):
    ind = individual_class(random.uniform(min_value, max_value) 
                           for _ in range(size))
    # we modify the instance to include the strategy in run-time.
    ind.strategy = strategy_class(random.uniform(min_strat, max_strat) for _ in range(size))
    return ind

#Para usar DEAP é necessário inicializar os valores dos objetos por meio do método register.
def execution():
  toolbox = base.Toolbox() 
  toolbox.register("individual", init_univariate_es_ind, 
                  creator.Individual, 
                  creator.Strategy,
                  search_space_dims, 
                  MIN_VALUE, MAX_VALUE, 
                  MIN_STRAT, MAX_STRAT)
  toolbox.register("population", tools.initRepeat, list, 
                  toolbox.individual)

  toolbox.register("mutate", tools.mutESLogNormal, c=1, indpb=0.1)

  toolbox.register("mate", tools.cxESBlend, alpha=0.1)
  #Neste exemplo estamos utilizando uma função da biblioteca de benchmarks do framework, mas você pode definir as sua própria função.
  #Por exemplo: def evaluate(individual):
  #                 return sum(individual)
  #toolbox.register("evaluate", evaluate)
  toolbox.register("evaluate", evalProblem)
  toolbox.register("select", tools.selBest)

  mu_es, lambda_es = 35,265

  pop = toolbox.population(n=mu_es)
  #retorna a melhor solução da população
  hof = tools.HallOfFame(1)

  #ferramenta para armazenar as estatísticas da evolução
  # pop_stats = tools.Statistics(key=copy.deepcopy)
  # pop_stats.register('pop', copy.deepcopy) # -- copia as próprias populações
  pop_stats = tools.Statistics(lambda ind: ind.fitness.values)
  pop_stats.register("avg", np.mean)
  pop_stats.register("std", np.std)
  pop_stats.register("min", np.min)
  pop_stats.register("max", np.max)

  #executa uma das 4 formas definidas de algoritmo no framework, nesse caso é o modelo de ES(mu,lambda)
  #é possível definir seu próprio algoritmo.    
  pop, logbook = algorithms.eaMuCommaLambda(pop, toolbox, mu=mu_es, lambda_=lambda_es, 
          cxpb=0.6, mutpb=0.1, ngen=300, stats=pop_stats, halloffame=hof, verbose=False)

  #plot_problem_controur(evalProblem, ((MIN_VALUE,MIN_VALUE), (MAX_VALUE,MAX_VALUE)), optimum=(0,0))
  #plot_individual(hof[0])

  print()
  print("-- Generation %i --" % logbook.select('min').index(min(logbook.select('min'))))
  print("-- Min %s" % min(logbook.select('min')))

  #pop_min_value.append(min(logbook.select('min')))  

  
  
  #plt.figure(2, figsize=(7, 4))
 #plt.plot(logbook.select('avg'), 'b-', label='Avg. fitness')
  #plt.fill_between(range(len(logbook)), logbook.select('max'), logbook.select('min'), facecolor='blue', alpha=0.47)
  #plt.fill_between(range(len(logbook)),logbook.select('min'), facecolor='red', alpha=0.47)
  #plt.plot(logbook.select('std'), 'm--', label='Std. deviation')
  #plt.legend(frameon=True)
  #plt.ylabel('Fitness'); plt.xlabel('Iterations');

  return min(logbook.select('min')),logbook.select('min'),logbook.select('std')


min_value = list()
population_best_fitness = list()
population_dv = list()
best_fitness = 999.
#mean_value = list()
#std_value = list()

for ex in range(1,31):
  min_fitness,pop_min_value, pop_dv = execution()
  min_value.append(min_fitness)

  if min_fitness < best_fitness:
    best_fitness = min_fitness 
    population_best_fitness = pop_min_value.copy()
    population_dv = pop_dv.copy()



generation = -1
epsilon = 0.0000001
for i in range (0, len(population_best_fitness) - 1):
  if(population_best_fitness[i] < population_best_fitness[i+1] and (population_best_fitness[i+1] - population_best_fitness[i]) > epsilon):
    generation = i+1

print("Melhor fitness {:.5f}".format(best_fitness))
print("Desvio padrão do fitness {:.5f}".format(np.std(min_value)))
print("Média do fitness {:.5f}".format(np.mean(min_value)))
print("Máximo do fitness {:.5f}".format(np.max(min_value)))
print ("Geração: ",generation)


#dp, media, maior, menor
print( "[{:.5f}, {:.5f}, {:.5f}, {:.5f}]".format(np.std(min_value),np.mean(min_value),np.max(min_value),best_fitness))
plt.clf()
plt.plot(range(1, len (population_best_fitness)+1), population_best_fitness, linewidth=3, color='red')
#plt.plot(population_dv, 'm--', label='Std. deviation')
#plt.plot(n_list, std_value, linewidth=3, color='blue', label='Desvio padrão do fitness')
plt.legend(loc='upper right')
plt.title('Estratégia Evolutiva', fontsize=19)
plt.xlabel('Geração')
plt.ylabel('Fitness')
plt.show()